In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip3 install pickle5

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
pip install pickle5

In [5]:
from glob import glob
import pickle5 as pickle
import numpy as np

ROOT_PATH = "../input/"

cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
splits = ["train", "test"]

def get_city_trajectories(city="palo-alto", split="train", normalized=False):
    if split=="train":
        f_in = ROOT_PATH + split + "/" + city + "_inputs"
        inputs = pickle.load(open(f_in, "rb"))
        inputs = np.asarray(inputs)

        outputs = None
    
        f_out = ROOT_PATH + split + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)
        
    elif split=="test":
        f_in = ROOT_PATH + "test-inputs" + "/" + city + "_inputs"
        inputs = pickle.load(open(f_in, "rb"))
        inputs = np.asarray(inputs)

        outputs = None
        
    return inputs, outputs

# class ArgoverseDataset(Dataset):
#     """Dataset class for Argoverse"""
#     def __init__(self, city: str, split:str, transform=None):
#         super(ArgoverseDataset, self).__init__()
#         self.transform = transform

#         self.inputs, self.outputs = get_city_trajectories(city=city, split=split, normalized=False)

#     def __len__(self):
#         return len(self.inputs)

#     def __getitem__(self, idx):

#         data = (self.inputs[idx], self.outputs[idx])
            
#         if self.transform:
#             data = self.transform(data)

#         return data

# # intialize a dataset
# count_train = 0
# for i in cities:
#     city = i  
#     split = 'train'
#     train_dataset  = ArgoverseDataset(city = city, split = split)
#     count_train += len(train_dataset)
# print("the number of training data:" + str(count_train))

# count_test = 0
# for i in cities:
#     city = i  
#     split = 'test'
#     test_dataset  = ArgoverseDataset(city = city, split = split)
#     count_test+= len(test_dataset)
# print("the number of training data:" + str(count_test))

In [6]:
# convert the original training document into trainable batches 
def get_batches(inp, batch_size):
    
    num_batches = int(len(inp) / (batch_size))
    
    X = inp[:num_batches*batch_size]
#     Y = np.zeros((len(X), 50))
    
    
    # generate trainable batches
    for i in range(0, num_batches*batch_size, batch_size):
        Xs = []
        Ys = []
        for j in range(i, i+batch_size):
            temp_x = X[j][:-1]
               
            temp_y = X[j][1:]
            
            Xs.append(temp_x)
            Ys.append(temp_y)
        
        Xs = np.array(Xs)
        Ys = np.array(Ys)

        yield torch.tensor(Xs).float(), torch.tensor(Ys).float()

In [7]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

class RNN_new_1(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=2):
        super(RNN_new_1, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.encode1 = nn.Linear(input_size, hidden_size)
        # input tensor - (batch, seq, tensor)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=n_layers, batch_first=True)
        self.bilstm = nn.LSTM(hidden_size, hidden_size//2, num_layers=n_layers, batch_first=True, bidirectional=True)
        
        self.encode2 = nn.Linear(input_size, hidden_size)
        
        self.encode3 = nn.Linear(hidden_size, hidden_size * 2)
        
        self.encode4 = nn.Linear(hidden_size * 2, hidden_size // 2)
        
        self.encode5 = nn.Linear(input_size, hidden_size // 2)
        
        self.encode6 = nn.Linear(hidden_size//2, output_size)
        
        
        self.decoder = nn.Linear(hidden_size, output_size)
        

        
    def forward(self, x_input, prev_hidden_and_cell, temperature=1.0):
        
        encode1 = self.encode1(x_input)

#         print(encode1.unsqueeze(1).shape)
        output, (hidden, cell) = self.lstm(encode1.unsqueeze(1), prev_hidden_and_cell)
        
        encode2 = self.encode2(x_input)
        bi_output, (hidden, cell) = self.lstm(encode2.unsqueeze(1), prev_hidden_and_cell)
#         print(bi_output.shape)
        f_output = torch.tanh(self.encode3(output + bi_output))
#         print(f_output.shape)
        f_output1 = torch.tanh(self.encode4(f_output))
#         print(f_output1.shape)
        f_output2 = self.encode5(x_input)
        
        f_output2 = f_output2.reshape(f_output2.shape[0], 1, f_output2.shape[1])

        output = self.encode6(f_output1 + f_output2)
        output = output.reshape(len(output), 2)
#         print(output.shape)

        return output, (hidden, cell)


    def init_state(self, batch_size):
        # return the initial hidden state and cell state
        return torch.zeros(self.n_layers, batch_size, self.hidden_size), torch.zeros(self.n_layers, batch_size, self.hidden_size)

In [8]:
def train_model(inp, n_epochs, hidden_size, batch_size, seq_size, lr, temperature):
    
    # (input_size/feature dim, num of characters, hidden_size, output_size, n_layers=1)
    model = RNN_new_1(2, hidden_size, 2, 1)
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    avg_loss_over_time = []
    for epoch in range(n_epochs):
        batches = get_batches(inp, batch_size)

        hidden, cell = model.init_state(batch_size)

        i = 0
        print("new epoch starts, currently at " + str(epoch) + " epoch")
        
        loss_avg = 0
        
        for X, Y in batches:
            i += 1

            # Reset all gradients
            optimizer.zero_grad()
            
            loss = 0
                         
            for i in range(seq_size-1):
                temp_x = X[:, i]
                temp_y = Y[:, i]
                output, (hidden, cell) = model.forward(temp_x.reshape(batch_size, 2), (hidden, cell), temperature)
#                 print(output)
#                 print(output.shape)
                loss += criterion(output, temp_y.reshape(batch_size, 2))
            
            hidden = hidden.detach()
            cell = cell.detach()
            
            loss.backward(retain_graph=True)
            optimizer.step()
            loss_avg += loss.item() / seq_size
            
        avg_loss_over_time.append(loss_avg / i)
        print("avg loss at " + str(i) + "th iteration - " + str(loss_avg / i))
        print("\n")  
    
    
    plt.plot(avg_loss_over_time)
    plt.xlabel("epoch num")
    plt.ylabel("avg_loss")
    return model

In [9]:
import time
import math

In [10]:
p_i, p_o = get_city_trajectories(city="palo-alto", split="train", normalized=True)

In [11]:
train_combined = np.concatenate([p_i, p_o], axis=1)

In [12]:
p_test_i, p_test_o = get_city_trajectories(city="palo-alto", split="test", normalized=True)

In [13]:
def normalize(train_combined):
    
    x_mean = np.mean(train_combined[:, :, 0].reshape(-1))
    x_std = np.std(train_combined[:, :, 0].reshape(-1))
    x_norm = ((train_combined[:, :, 0].reshape(-1) - x_mean) / x_std).reshape(train_combined.shape[0], train_combined.shape[1], 1)

    y_mean = np.mean(train_combined[:, :, 1].reshape(-1))
    y_std = np.std(train_combined[:, :, 1].reshape(-1))
    y_norm = ((train_combined[:, :, 1].reshape(-1) - y_mean) / y_std).reshape(train_combined.shape[0], train_combined.shape[1], 1)

    normalized = np.concatenate([x_norm, y_norm], axis=2)
    
    return normalized

In [14]:
train_combined_norm = normalize(train_combined)

In [15]:
# (inp, n_epochs, hidden_size, batch_size, seq_size, lr, temperature)
start = time.time()
print("Starts training process")
model_1 = train_model(train_combined, 20, 128, 500, 110, 0.05, 1.5)
end = time.time()
print("Ends training process, total time - " + str(end - start))

In [ ]:
x_global_mean = np.mean(train_combined[:, :, 0].reshape(-1))
x_global_std = np.std(train_combined[:, :, 0].reshape(-1))
y_global_mean = np.mean(train_combined[:, :, 1].reshape(-1))
y_global_std = np.std(train_combined[:, :, 1].reshape(-1))

In [ ]:
def predict(model, inp, predict_length, normalize_test_data=False):
    
    if normalize_test_data:
        x_mean = np.mean(inp[:, 0].reshape(-1))
        x_std = np.std(inp[:, 0].reshape(-1))
        y_mean = np.mean(inp[:, 1].reshape(-1))
        y_std = np.std(inp[:, 1].reshape(-1))
        x_norm = ((inp[:, 0].reshape(-1) - x_mean) / x_std).reshape(inp.shape[0], 1)
        y_norm = ((inp[:, 1].reshape(-1) - y_mean) / y_std).reshape(inp.shape[0], 1)

        norm = np.concatenate([x_norm, y_norm], axis=1)
    
        inp = norm

    
    hidden, cell = model.init_state(1)
    
    
    input_batch = get_batches(inp.reshape(1,len(inp),2), 1)

    X, _ = next(input_batch)
    
    # build upon our hidden and cell state based on the first seq_size-1 positions
    for i in range(len(inp)-1):
        temp_x = X[:, i]
#         print(temp_x)
        output, (hidden, cell) = model.forward(temp_x, (hidden, cell))

#         print(output)

    prev = inp[-1]
#     print(prev)
    predict = []
    # make the prediction character by character
    for _ in range(predict_length):
        
        if _ == 0:
            temp_x = torch.tensor([prev]).float()
        else:
            temp_x = prev
       
        output, (hidden, cell) = model.forward(temp_x, (hidden, cell))

#         output = defined_softmax(output.reshape(1, 1, 61), temperature)

        prev = output
        
        if normalize_test_data:
            x_pred = (prev[0][0].item()) * x_std + x_mean
            y_pred = (prev[0][1].item()) * y_std + y_mean
        else:
            x_pred = (prev[0][0].item())
            y_pred = (prev[0][1].item())
            
        predict.append([x_pred, y_pred])
    
    return predict

In [ ]:
p_o[0]

In [ ]:
predict(model_1, p_i[0], 60, False)

In [ ]:
palo_df = pd.DataFrame()
for i in range(len(p_test_i)):
    
    pred = predict(model_1, p_test_i[i], 60, False)
    print("predicts for " + str(i) + " is done")
    temp_dict = {}
    for j in range(0, 120, 2):
        col_x = 'v' + str(j)
        col_y = 'v' + str(j+1)
        row = j // 2
        temp_dict[col_x] = pred[row][0]
        temp_dict[col_y] = pred[row][1]
    
    temp_idx_name = str(i) + "_" + "palo-alto"
    temp_final = pd.DataFrame(temp_dict, index=[temp_idx_name])
    palo_df = pd.concat([palo_df, temp_final])

In [ ]:
palo_df.reset_index(level=0).rename(columns={"index": "ID"}).to_csv("model2_palo_df.csv", index=False)

In [ ]:
df_actual_path = "../input/pred-actual/palo-alto_actual.csv"
df_pred_path = "../input/pred1/model1_palo_df.csv"

def actual_error(df_actual_path, df_pred_path):
    actual = pd.read_csv(df_actual_path)    
    actual_array = actual.iloc[:, 1:].to_numpy()
    
    pred1 = pd.read_csv(df_pred_path)
    pred1_array = pred1.iloc[:, 1:].to_numpy()
    
    return np.sum((actual_array - pred1_array)**2) / len(actual_array)

actual_error(df_actual_path, df_pred_path)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
 
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
 


In [ ]:
p_i.shape

In [ ]:
p_i[:, :, 0].shape

In [ ]:
p_o[:, 0, 0].shape

In [ ]:
regressor.fit(p_i[:, :, 0], p_o[:, 0, 0])

In [ ]:
regressor.predict(p_i[0,:, 0].reshape(1, -1))[0]

In [ ]:
def preprocess_data(inp):
    
    final_feature = inp[:, :50, 0]
    final_target = inp[:, 50, 0]
    for i in range(1, 60):
        temp_x = inp[:, i:50+i, 0]
        temp_y = inp[:, 50+i, 0]
        final_feature = np.concatenate([final_feature, temp_x], axis=0)
        final_target = np.concatenate([final_target, temp_y], axis=0)
    
    return final_feature, final_target
    

In [ ]:
X, y = preprocess_data(train_combined)

In [ ]:
regressor.fit(X, y)

In [ ]:
def predict_regressor(model, inp, pred_len):
    
    result = []
    curr = inp[:, 0]
    pred = model.predict(curr.reshape(1, -1))[0]
    result.append(pred)
    for i in range(1, pred_len):
        curr = np.append(curr[1:], pred)
        pred = model.predict(curr.reshape(1, -1))[0]
        result.append(pred)
    
    return result
    